## Image Datasets
- https://www.kaggle.com/airplane2230/apparel-image-dataset-2 // Apparel image dataset 2


In [ ]:
## 1. Library Import 및 Data Load

In [4]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_PATH = 'data/multi_label/'
train_df = pd.read_csv(DATA_PATH + 'train.csv', index_col=0)
val_df = pd.read_csv(DATA_PATH + 'val.csv', index_col=0)
test_df = pd.read_csv(DATA_PATH + 'test.csv', index_col=0)

# class column들
class_col = list(train_df.columns[1:])
print(class_col)

batch_size = 16


train_df.head()

,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes
0,./clothes_dataset\blue_shorts\256d854b55ac32ea...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,./clothes_dataset\red_pants\584f778aece14f07c2...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,./clothes_dataset\green_pants\ec543ca241cefb2b...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,./clothes_dataset\brown_shorts\c8db9e0f7010592...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,./clothes_dataset\white_dress\551373c80717c5b0...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## 2-1. Image Data Generator 정의

In [5]:
# 이미지 제너레이터를 정의합니다.
train_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size

## 2-2. Generator - flow from dataframe 생성

In [ ]:
# Make Generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory = DATA_PATH, # os.getcwd() 기반 ~
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory = DATA_PATH,
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='other',
    batch_size=batch_size,
    shuffle=True
)

## 3. Model 생성

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()
# 입력 데이터의 형태를 꼭 명시해야 합니다.
model.add(Flatten(input_shape = (112, 112, 3))) # (112, 112, 3) -> (112 * 112 * 3)
model.add(Dense(128, activation = 'relu')) # 128개의 출력을 가지는 Dense 층
model.add(Dense(64, activation = 'relu')) # 64개의 출력을 가지는 Dense 층
model.add(Dense(11, activation = 'sigmoid')) # 11개의 출력을 가지는 신경망

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', metrics = ['acc'])

In [26]:
model.fit(train_generator,
         steps_per_epoch=get_steps(len(train_df), batch_size),
         validation_data = val_generator,
         validation_steps=get_steps(len(val_df), batch_size),
         epochs = 10)

Epoch 1/10
349/349 [==============================] - 23s 67ms/step - loss: 0.5599 - acc: 0.3473 - val_loss: 0.3117 - val_acc: 0.5090
Epoch 2/10
349/349 [==============================] - 23s 65ms/step - loss: 0.3036 - acc: 0.4303 - val_loss: 0.3093 - val_acc: 0.4023
Epoch 3/10
349/349 [==============================] - 23s 66ms/step - loss: 0.2620 - acc: 0.4665 - val_loss: 0.2696 - val_acc: 0.4504
Epoch 4/10
349/349 [==============================] - 23s 66ms/step - loss: 0.2275 - acc: 0.5197 - val_loss: 0.2587 - val_acc: 0.5947
Epoch 5/10
349/349 [==============================] - 23s 67ms/step - loss: 0.2129 - acc: 0.5332 - val_loss: 0.2158 - val_acc: 0.5437
Epoch 6/10
349/349 [==============================] - 23s 66ms/step - loss: 0.1966 - acc: 0.5579 - val_loss: 0.2459 - val_acc: 0.5625
Epoch 7/10
349/349 [==============================] - 23s 67ms/step - loss: 0.1956 - acc: 0.5563 - val_loss: 0.2081 - val_acc: 0.5709
Epoch 8/10
349/349 [==============================] - 23s 67ms